In [ ]:
import os

os.mkdir('/content/sample_data/llms')
os.chdir('/content/sample_data/llms')

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 4485, done.
remote: Counting objects: 100% (2076/2076), done.
remote: Compressing objects: 100% (359/359), done.
remote: Total 4485 (delta 1905), reused 1778 (delta 1717), pack-reused 2409
Receiving objects: 100% (4485/4485), 3.85 MiB | 10.56 MiB/s, done.
Resolving deltas: 100% (3042/3042), done.
/content/sample_data/llms/llama.cpp


In [ ]:
#!mkdir build
#%cd build
#!cmake ..
#!cmake --build . --config Release

!make

I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I CXXFLAGS: -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I LDFLAGS:  
I CC:       cc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
I CXX:      g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0

cc  -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS   -c ggml.c -o ggml.o
g++ -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native 

In [ ]:
!pwd

/content/sample_data/llms/llama.cpp


In [ ]:
# Make sure you have git-lfs installed (https://git-lfs.com)
!git lfs install
!git clone https://huggingface.co/openlm-research/open_llama_3b

Updated git hooks.
Git LFS initialized.
Cloning into 'open_llama_3b'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), 6.42 KiB | 939.00 KiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model.bin

See: `git lfs help smudge` for more details.


In [ ]:
# obtain the original LLaMA model weights and place them in ./models
import shutil
shutil.move("./open_llama_3b", "./models/open_llama_3b")

# install Python dependencies
!python3 -m pip install -r requirements.txt

# convert the 7B model to ggml FP16 format
!python3 convert.py models/open_llama_3b

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.0 which is incompatible.
seaborn 0.12.2 requires numpy!=1.24.0,>=1.17, but you have numpy 1.24.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.0 which is incompatible.
Loading model file models/open_llama_3b/pytorch_model.bin
Loading vocab file models/open_llama_3b/tokenizer.model
params: n_vocab:32000 n_embd:3200 n_mult:240 n_head:32 n_layer:26
Writing vocab...
[  1/237] Writing tensor tok_embeddin

In [ ]:
!ls

build-info.h		 ggml-metal.m	   Package.swift
build.zig		 ggml-metal.metal  perplexity
CMakeLists.txt		 ggml.o		   pocs
common.o		 ggml-opencl.cpp   prompts
convert-lora-to-ggml.py  ggml-opencl.h	   quantize
convert-pth-to-ggml.py	 k_quants.c	   quantize-stats
convert.py		 k_quants.h	   README.md
docs			 k_quants.o	   requirements.txt
embd-input-test		 libembdinput.so   scripts
embedding		 LICENSE	   server
examples		 llama.cpp	   SHA256SUMS
flake.lock		 llama.h	   simple
flake.nix		 llama.o	   spm-headers
ggml.c			 llama-util.h	   tests
ggml-cuda.cu		 main		   train-text-from-scratch
ggml-cuda.h		 Makefile	   vdot
ggml.h			 media
ggml-metal.h		 models


In [ ]:
# quantize the model to 4-bits (using q4_0 method)
!./quantize ./models/open_llama_3b/ggml-model-f16.bin ./models/open_llama_3b/ggml-model-q4_0.bin q4_0

# run the inference
!./main -m ./models/open_llama_3b/ggml-model-q4_0.bin -n 128

main: build = 800 (481f793)
main: quantizing './models/open_llama_3b/ggml-model-f16.bin' to './models/open_llama_3b/ggml-model-q4_0.bin' as Q4_0
llama.cpp: loading model from ./models/open_llama_3b/ggml-model-f16.bin
llama.cpp: saving model to ./models/open_llama_3b/ggml-model-q4_0.bin
[   1/ 237]                tok_embeddings.weight -     3200 x 32000, type =    f16, quantizing .. size =   195.31 MB ->    54.93 MB | hist: 0.036 0.016 0.025 0.039 0.057 0.077 0.096 0.111 0.117 0.111 0.096 0.077 0.057 0.039 0.025 0.021 
[   2/ 237]                          norm.weight -             3200, type =    f32, size =    0.012 MB
[   3/ 237]                        output.weight -     3200 x 32000, type =    f16, quantizing .. size =   195.31 MB ->    54.93 MB | hist: 0.036 0.015 0.024 0.037 0.055 0.076 0.097 0.114 0.121 0.114 0.097 0.076 0.055 0.037 0.024 0.020 
[   4/ 237]         layers.0.attention.wq.weight -     3200 x  3200, type =    f16, quantizing .. size =    19.53 MB ->     5.49 MB | hi

In [ ]:
!pip install llama-cpp-python
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.68-cp310-cp310-linux_x86_64.whl size=265311 sha256=9dec9d8a0840800c7d1eca00d206a28856caae5c1a0c86671fc321f4e5c6dea7
  Stored in directory: /root/.cache/pip/wheels/df/f2/fb/b8153a244ace60fa4759cbd3d4881a2132b71e0e894ed6f29b
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.utilities import SerpAPIWrapper

In [ ]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

In [ ]:
!ls

build-info.h		 ggml-metal.m	   Package.swift
build.zig		 ggml-metal.metal  perplexity
CMakeLists.txt		 ggml.o		   pocs
common.o		 ggml-opencl.cpp   prompts
convert-lora-to-ggml.py  ggml-opencl.h	   quantize
convert-pth-to-ggml.py	 k_quants.c	   quantize-stats
convert.py		 k_quants.h	   README.md
docs			 k_quants.o	   requirements.txt
embd-input-test		 libembdinput.so   scripts
embedding		 LICENSE	   server
examples		 llama.cpp	   SHA256SUMS
flake.lock		 llama.h	   simple
flake.nix		 llama.o	   spm-headers
ggml.c			 llama-util.h	   tests
ggml-cuda.cu		 main		   train-text-from-scratch
ggml-cuda.h		 Makefile	   vdot
ggml.h			 media
ggml-metal.h		 models


In [ ]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="./models/open_llama_3b/ggml-model-q4_0.bin", callback_manager=callback_manager, verbose=True
)




AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "What would be a good company name for a company that makes colorful socks"

llm_chain.run(question)

Llama.generate: prefix-match hit




*

*Company name: First we choose a name for the company. A company name is
the unique name of the organization, with no spaces between
the words and no spaces after the last word. There are different rules
for choosing a company name. The first is that the name must
be in a language which is commonly understood around the world
or at least commonly understood in your state or country. A second
rule is that you cannot use the last name of an existing
company that already exists. Let's take these rules and break them down.
First, if the company name can be easily translated to English then it is in compliance with the first rule because it is a foreign language. It is also in compliance with the second rule if the company name is an existing company that does not already exist (other than as part of the name of another company). In this case, the company name is already being used.
A company name can be made up or can be a combination of words. There are two approaches. Either it can 

"\n\n*\n\n*Company name: First we choose a name for the company. A company name is\nthe unique name of the organization, with no spaces between\nthe words and no spaces after the last word. There are different rules\nfor choosing a company name. The first is that the name must\nbe in a language which is commonly understood around the world\nor at least commonly understood in your state or country. A second\nrule is that you cannot use the last name of an existing\ncompany that already exists. Let's take these rules and break them down.\nFirst, if the company name can be easily translated to English then it is in compliance with the first rule because it is a foreign language. It is also in compliance with the second rule if the company name is an existing company that does not already exist (other than as part of the name of another company). In this case, the company name is already being used.\nA company name can be made up or can be a combination of words. There are two approaches. 